# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League
import time
import pickle
import uuid

from python import constants, functions

# Read the ESPN API to add to Dataset

### Fetch all League and Matchup data via api and save to pickle

In [130]:
leagues = []
matchups = []

for year in range(2019,2025):
    league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)
    leagues.append(
        {
            'Year':year,
            'League':league
        }
    )

    season_length = 17
    if year in [2019,2020]:
        season_length = 16

    for week in range(1,season_length + 1):
        box_scores = league.box_scores(week)
        matchups.append(
            {
                'Year':year,
                'Week':week,
                'Box Scores':box_scores
            }
        )

        time.sleep(3)
        
data = {
    'Leagues':leagues,
    'Box Scores':matchups
}

In [140]:
with open('espn-api-2019-2024.pkl', 'wb') as file:
    pickle.dump(data, file)

In [22]:
with open('database/espn-data.pkl', 'rb') as file:
    data_new = pickle.load(file)

In [ ]:
import requests
import json

year = 2024

url = f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{constants.LEAGUE_ID}'

filters = {
    "transactions": {
        "filterType": {
            "value": [
                'DRAFT',
                'FREEAGENT',
                'FUTURE_ROSTER',
                'RETRO_ROSTER',
                'ROSTER',
                'TRADE_ACCEPT',
                'TRADE_ERROR',
                'TRADE_PROPOSAL',
                'TRADE_UPHOLD',
                'TRADE_VETO',
                'WAIVER',
                'WAIVER_ERROR'
            ]
        }
    }
}

headers = {
    "x-fantasy-filter": json.dumps(filters)
}

cookies = {
    'espn_s2':constants.ESPN_S2,
    'SWID':constants.SWID
}

params = {
    'view':'mTransactions2',
    'scoringPeriodId':8
}

r = requests.get(url, params=params, headers=headers, cookies=cookies)
data = r.json()


In [215]:
for record in data.get('transactions'):
    print(
        record['id'],
        record['items'][0]['playerId'],
        record['proposedDate'],
        record['executionType'],
        record['type'],
        record['status']
    )

3af2bcf0-ef76-45c1-9deb-cd8ba2e0812c 2985659 1729828438622 EXECUTE FREEAGENT EXECUTED
d554bf8b-ccab-42b5-a67f-70d8fa3929a1 14881 1729671026271 PROCESS WAIVER EXECUTED
91325026-6d46-42aa-b561-81bd2c11ce3c 4428209 1729671026271 PROCESS WAIVER EXECUTED
4291610c-7231-4b9a-be96-4e0f1642ed1e 4567104 1729671026271 PROCESS WAIVER EXECUTED
1ae7ea58-693e-4bc8-b401-82bd0cec2264 2977187 1729512780439 EXECUTE FUTURE_ROSTER EXECUTED
6b36c8c7-7196-4bb5-a293-9a76cec54842 4047365 1729560521948 EXECUTE FUTURE_ROSTER EXECUTED
fb84c216-1b02-49a0-9671-c0bba298d6a0 4430834 1729613926068 CANCEL WAIVER CANCELED
3966c1fd-b052-4f89-b7c7-4da62fc3c91b 4428209 1729626780073 CANCEL WAIVER CANCELED
cc990ade-0989-4159-9cb2-c2cedb0d9222 4430834 1729671026271 PROCESS WAIVER FAILED_INVALIDPLAYERSOURCE
d3dad0de-8056-4e17-a422-f64518b6b3df 4430834 1729671026271 PROCESS WAIVER FAILED_INVALIDPLAYERSOURCE
fe3b51c3-e283-4e71-a1b6-243cebacfdf3 4635008 1729568783495 EXECUTE FUTURE_ROSTER EXECUTED
7ff5b480-6e14-4f76-9e12-da838db

In [32]:
espn_api.football.constant.TRANSACTION_TYPES

{'DRAFT',
 'FREEAGENT',
 'FUTURE_ROSTER',
 'RETRO_ROSTER',
 'ROSTER',
 'TRADE_ACCEPT',
 'TRADE_DECLINE',
 'TRADE_ERROR',
 'TRADE_PROPOSAL',
 'TRADE_UPHOLD',
 'TRADE_VETO',
 'WAIVER',
 'WAIVER_ERROR'}

In [22]:
league = League(league_id=constants.LEAGUE_ID, year=2023, espn_s2=constants.ESPN_S2, swid=constants.SWID)
data_transactions = []
for week in range(17):
    try:
        week_transactions = league.transactions(week, types={
        'DRAFT',
        'FREEAGENT',
        'FUTURE_ROSTER',
        'RETRO_ROSTER',
        'ROSTER',
        'TRADE_ACCEPT',
        'TRADE_ERROR',
        'TRADE_PROPOSAL',
        'TRADE_UPHOLD',
        'TRADE_VETO',
        'WAIVER',
        'WAIVER_ERROR'
        })
    except:
        continue
    for trans in week_transactions:
        data_transactions.append(trans)
        
for i in range(len(data_transactions)):
    if 'TRADE' in data_transactions[i].type:
        print(i, data_transactions[i].items, data_transactions[i].type, data_transactions[i].status)

206 [TRADE Skyy Moore, TRADE Ezekiel Elliott] TRADE_PROPOSAL PENDING
222 [TRADE Skyy Moore, TRADE Ezekiel Elliott] TRADE_PROPOSAL CANCELED
316 [TRADE Gus Edwards, TRADE Trevor Lawrence, TRADE Trey McBride, TRADE Lamar Jackson] TRADE_ACCEPT EXECUTED


In [31]:
league = League(league_id=constants.LEAGUE_ID, year=2019, espn_s2=constants.ESPN_S2, swid=constants.SWID)
print({
    'first_week':league.firstScoringPeriod,
    'last week':league.finalScoringPeriod,
    'seasons':league.previousSeasons
})

{'first_week': 1, 'last week': 16, 'seasons': [2018]}


In [277]:
df = pd.DataFrame([vars(record) for record in data_transactions])
df
df.loc[(df['type'] == 'TRADE_ACCEPT') | (df['type'] == 'TRADE_PROPOSAL')]
# df['type'].unique()
# df.loc[(df['status'] == 'PENDING')]

,team,type,status,scoring_period,date,bid_amount,items
3,Team(i like the cowboys),TRADE_PROPOSAL,CANCELED,1,NaN,0,"[TRADE Rome Odunze, TRADE Ezekiel Elliott]"
54,Team(McBridesmaids),TRADE_PROPOSAL,CANCELED,1,NaN,0,"[TRADE George Pickens, TRADE Brian Robinson Jr..."
91,Team(McBridesmaids),TRADE_PROPOSAL,CANCELED,1,NaN,0,"[TRADE Davante Adams, TRADE Marvin Harrison Jr.]"
176,Team(i like the cowboys),TRADE_PROPOSAL,PENDING,1,NaN,0,"[TRADE Rome Odunze, TRADE Ezekiel Elliott]"
221,Team(Michaels Avg. Team),TRADE_PROPOSAL,CANCELED,2,NaN,0,"[TRADE Brandin Cooks, TRADE Dalton Kincaid, TR..."
223,Team(Michaels Avg. Team),TRADE_PROPOSAL,PENDING,2,NaN,0,"[TRADE Brandin Cooks, TRADE Dalton Kincaid, TR..."
463,Team(Pukie Bear),TRADE_ACCEPT,EXECUTED,14,1.733213e+12,0,"[TRADE Alexander Mattison, TRADE Jameson Willi..."


In [280]:
league.player_map.get(3045147)

'James Conner'

In [180]:
efr = espn_api.requests.espn_requests.EspnFantasyRequests(
    sport='nfl',
    year=2024,
    league_id=constants.LEAGUE_ID,
    cookies=cookies
)

data2 = efr.league_get(params=params, headers=headers)
for record in data2.get('transactions'):
    print(
        record['id'],
        record['executionType'],
        record['type'],
        record['status']
    )


3af2bcf0-ef76-45c1-9deb-cd8ba2e0812c EXECUTE FREEAGENT EXECUTED
d554bf8b-ccab-42b5-a67f-70d8fa3929a1 PROCESS WAIVER EXECUTED
91325026-6d46-42aa-b561-81bd2c11ce3c PROCESS WAIVER EXECUTED
4291610c-7231-4b9a-be96-4e0f1642ed1e PROCESS WAIVER EXECUTED
fb84c216-1b02-49a0-9671-c0bba298d6a0 CANCEL WAIVER CANCELED
3966c1fd-b052-4f89-b7c7-4da62fc3c91b CANCEL WAIVER CANCELED
cc990ade-0989-4159-9cb2-c2cedb0d9222 PROCESS WAIVER FAILED_INVALIDPLAYERSOURCE
d3dad0de-8056-4e17-a422-f64518b6b3df PROCESS WAIVER FAILED_INVALIDPLAYERSOURCE
d0cb7814-ef14-45b4-bff0-65c48581fdec CANCEL WAIVER CANCELED
609f90f0-f9c1-479f-ab6a-57ab792be3d0 EXECUTE FREEAGENT EXECUTED
1c675970-dc54-4b15-ad73-269d288aa865 PROCESS WAIVER EXECUTED
a1b4fdc4-1dc8-4bfd-b1c5-958c7124f33a PROCESS WAIVER FAILED_INVALIDPLAYERSOURCE
0ea8d5e5-f403-4c5e-941f-59b65d398c66 EXECUTE FREEAGENT EXECUTED
fab26119-7927-47f1-aaf5-bb4d4e762554 PROCESS WAIVER EXECUTED
3955fc95-faee-4ed2-97c7-e02bfb48f03b EXECUTE FREEAGENT EXECUTED
a98317dc-757b-4bfa-8f2

In [84]:
data_teams = []
data_drafts = []
data_players = []

leagues = data_new['Leagues']
for record in leagues:
    league = record['League']
    year = record['Year']

    for player in league.player_map.keys():
        if type(player) == int:
            continue
        data_players.append(player)

    for pick in league.draft:
        team_name = pick.team.owners[0]['firstName']
        if team_name == 'The':
            team_name = 'Klapp'
        elif team_name == 'Noah ':
            team_name = 'Noah'

        data_drafts.append(
            {
                'draft_pick_id':str(uuid.uuid4()),
                'player_id':str(uuid.uuid5(namespace=NAMESPACE, name=pick.playerName)),
                'team_id':str(uuid.uuid5(namespace=NAMESPACE, name=team_name)),
                'year':year,
                'round':pick.round_num,
                'pick':pick.round_pick
            }
        )

    for team in league.teams:
        team_name = team.owners[0]['firstName']
        if team_name == 'The':
            team_name = 'Klapp'
        elif team_name == 'Noah ':
            team_name = 'Noah'

        data_teams.append(team_name)

data_teams = [{'team_id':str(uuid.uuid5(namespace=NAMESPACE, name=team)), 'team_name':team} for team in list(set(data_teams))]
df_teams = pd.DataFrame(data_teams)

data_players = [{'player_id':str(uuid.uuid5(namespace=NAMESPACE, name=player)), 'player_name':player} for player in list(set(data_players))]
df_players = pd.DataFrame(data_players)

df_drafts = pd.DataFrame(data_drafts)

### Create Matchups + Games + Player Games + Players Dataframes

In [83]:
data_matchups = []
data_games = []
data_player_games = []
data_players = []

box_scores = data_new['Box Scores']
for box_score in box_scores:
    year = box_score['Year']
    week = box_score['Week']
    matchups = box_score['Box Scores']

    for matchup in matchups:
        matchup_id = str(uuid.uuid4())

        home_team = matchup.home_team.owners[0]['firstName']
        if home_team == 'The':
            home_team = 'Klapp'
        elif home_team == 'Noah ':
            home_team = 'Noah'

        if matchup.away_team == 0:
            away_team = 'Bye'
        else:
            away_team = matchup.away_team.owners[0]['firstName']
        if away_team == 'The':
            away_team = 'Klapp'
        elif away_team == 'Noah ':
            away_team = 'Noah'

        data_matchups.append(
            {
                'matchup_id':matchup_id,
                'year':year,
                'week':week,
                'matchup_type':matchup.matchup_type,
                'playoff_flag':matchup.is_playoff,
                'home_team_id':str(uuid.uuid5(NAMESPACE, name=home_team)),
                'home_score':matchup.home_score,
                'away_team_id':str(uuid.uuid5(NAMESPACE, name=away_team)),
                'away_score':matchup.away_score
            }
        )

        home_game_id = str(uuid.uuid4())
        data_games.append(
            {
                'game_id':home_game_id,
                'matchup_id':matchup_id,
                'team_id':str(uuid.uuid5(NAMESPACE, name=home_team)),
                'score':matchup.home_score,
                'opp_score':matchup.away_score,
                'win_flag':int(matchup.home_score > matchup.away_score),
                'margin':round(matchup.home_score - matchup.away_score, 2)
            }
        )

        away_game_id = str(uuid.uuid4())
        data_games.append(
            {
                'game_id':away_game_id,
                'matchup_id':matchup_id,
                'team_id':str(uuid.uuid5(NAMESPACE, name=away_team)),
                'score':matchup.away_score,
                'opp_score':matchup.home_score,
                'win_flag':int(matchup.away_score > matchup.home_score),
                'margin':round(matchup.away_score - matchup.home_score, 2)
            }
        )

        for player in matchup.home_lineup:
            data_player_games.append(
                {
                    'player_game_id':str(uuid.uuid4()),
                    'matchup_id':matchup_id,
                    'game_id':home_game_id,
                    'team_id':str(uuid.uuid5(NAMESPACE, name=home_team)),
                    'player_id':str(uuid.uuid5(NAMESPACE, name=player.name)),
                    'points':player.points,
                    'slot_position':player.slot_position,
                    'active_status':player.active_status,
                    'bye_week_flag':player.on_bye_week
                }
            )

        for player in matchup.away_lineup:
            data_player_games.append(
                {
                    'player_game_id':str(uuid.uuid4()),
                    'game_id':away_game_id,
                    'matchup_id':matchup_id,
                    'team_id':str(uuid.uuid5(NAMESPACE, name=away_team)),
                    'player_id':str(uuid.uuid5(NAMESPACE, name=player.name)),
                    'points':player.points,
                    'slot_position':player.slot_position,
                    'active_status':player.active_status,
                    'bye_week_flag':player.on_bye_week
                }
            )

        for player in (matchup.home_lineup + matchup.away_lineup):
            data_players.append(player.name)


df_matchups = pd.DataFrame(data_matchups)
df_games = pd.DataFrame(data_games)
df_player_games = pd.DataFrame(data_player_games)

In [11]:
year, week = 2023, 7

lineup_data = constants.PLAYER_MATCHUP_DATA.loc[(constants.PLAYER_MATCHUP_DATA['Year'] == year) & (constants.PLAYER_MATCHUP_DATA['Week'] == week)]
lineup_data.index = ('row-' + lineup_data['Home Team'].astype(str) + 'vs' + lineup_data['Away Team'].astype(str) + '-' + lineup_data['Position'].astype(str)).str.lower()
lineup_data['Matchup Lookup'] = lineup_data['Home Team'].astype(str) + 'vs' + lineup_data['Away Team'].astype(str)
lineup_table = functions.df_to_table(lineup_data[['Home Team','Home Player','Home Player Points','Position','Away Player Points','Away Player','Away Team']], index_to_id=True, table_id='lineup-table')
lineup_div = functions.content_container(title='Lineups', content=lineup_table)

lineup_title = lineup_div.get(h2)[0]
lineup_select = select([option('All', value='all')] + [option(teams.split('vs')[0] + ' vs ' + teams.split('vs')[1], value=teams.lower()) for teams in lineup_data['Matchup Lookup'].unique()],
                        _id='lineup-filter',
                        onchange="tableFilter('lineup-filter', 'lineup-table')")
lineup_title.add(lineup_select)

print(lineup_div)

<div class="content-container">
  <h2>Lineups
    <select id="lineup-filter" onchange="tableFilter('lineup-filter', 'lineup-table')">
      <option value="all">All</option>
      <option value="cartervsdante">Carter vs Dante</option>
      <option value="ethanvsandrew">Ethan vs Andrew</option>
      <option value="mcgwirevsmichael">McGwire vs Michael</option>
      <option value="noahvskevin">Noah vs Kevin</option>
      <option value="tylervsharis">Tyler vs Haris</option>
      <option value="zachvsnathan">Zach vs Nathan</option>
    </select>
  </h2>
  <div class="scroll-table">
    <table id="lineup-table">
      <thead>
        <tr>
          <th>Home Team</th>
          <th>Home Player</th>
          <th>Home Player Points</th>
          <th>Position</th>
          <th>Away Player Points</th>
          <th>Away Player</th>
          <th>Away Team</th>
        </tr>
      </thead>
      <tbody>
        <tr id="row-cartervsdante-qb"><td>Carter</td><td>Joshua Dobbs</td><td>16.14</td>

/var/folders/b5/lkkh5rx9771c1mbt1ckv105w0000gn/T/ipykernel_81519/650037926.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lineup_data['Matchup Lookup'] = lineup_data['Home Team'].astype(str) + 'vs' + lineup_data['Away Team'].astype(str)


In [ ]:
lineup_data['Matchup Lookup'].unique()

array(['CartervsDante', 'EthanvsAndrew', 'McGwirevsMichael',
       'NoahvsKevin', 'TylervsHaris', 'ZachvsNathan'], dtype=object)